In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from ds_discovery import SyntheticBuilder, ModelsBuilder
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## synthetic datasets

In [3]:
from ds_discovery import SyntheticBuilder

In [4]:
sb = SyntheticBuilder.from_memory()
sb.add_connector_persist('source', uri_file='sourcefile.parquet')
sb.add_connector_persist('other', uri_file='otherfile.parquet')

In [5]:
dfA = pd.DataFrame()
dfA['ref'] = sb.tools.get_number(10_000, 99_000, precision=0, at_most=1, size=1000)
dfA['A'] = sb.tools.get_category(selection=[1, 0], relative_freq=[9.1], size=1000)
dfB = pd.DataFrame()
dfB['ref'] = dfA['ref'].sample(frac=1)
dfB['B'] = sb.tools.get_category(selection=[1, 0], relative_freq=[8,1], size=1000)


In [6]:
sb.save_canonical('source', dfA)
sb.save_canonical('other', dfB)

## difference

In [7]:
from ds_discovery import Wrangle

In [8]:
wr = Wrangle.from_memory()

In [9]:
wr.set_source('sourcefile.parquet')
sb.add_connector_persist('other', uri_file='otherfile.parquet')

In [10]:
df = wr.load_source_canonical()
df.columns

Index(['ref', 'A'], dtype='object')

In [12]:
other = sb.load_canonical('other')
other.columns

Index(['ref', 'B'], dtype='object')

In [13]:
other = other.to_dict()

In [14]:
df = wr.tools.model_merge(df, other=other, on='ref')
df.shape

(1000, 3)